# 프로젝트4 : Scikit-learn의 Toy Dataset 활용



##<목차>
Step 1. 데이터 다운로드   
Step 2. 데이터 읽어오기   
Step 3. 데이터 정제   
- 특수문자 제거   
- 정규표현식 데이터 정제   
- [번외]기타 실험   

Step 4. 평가 데이터셋 분리   
- 토크나이저 생성   
- 텐서플로우 활용 데이터 분리   

Step 5. 인공지능 만들기   
- 최종 모델 학습(🎉🎉완벽 성공🎉🎉)
- validation loss 2.2 이하를 위한 노력의 흔적

Step 6. 작문 평가   
Step 7. 회고   
Step 8. Reference

##Step 1. 데이터 다운로드

In [ ]:
! mkdir -p ~/content/lyricist/models
! ln -s ~/data ~/content/lyricist/data

##Step 2. 데이터 읽어오기

In [72]:
import glob
import os

txt_file_path = '/content/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Can we forget about the things I said when I was drunk...', "I didn't mean to call you that", "I can't remember what was said"]


##Step 3. 데이터 정제
###**특수문자 제거**

In [73]:
#정규표현식 정제
import os, re 

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


###**정규표현식 데이터 정제**

In [74]:
# 정규표현식 데이터 정제

corpus = []  #정제한 데이터를 저장하기 위함

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    if preprocessed_sentence.count(' ') > 15 : continue
    corpus.append(preprocessed_sentence)

    #잘 적용되었는지 확인
corpus[140:150]

['<start> the unsuspecting victim of darkness in the valley <end>',
 '<start> we can live like jack and sally if we want <end>',
 '<start> where you can always find me <end>',
 '<start> and we ll have halloween on christmas <end>',
 '<start> and in the night we ll wish this never ends <end>',
 '<start> we ll wish this never ends i miss you i miss you <end>',
 '<start> i miss you i miss you where are you and i m so sorry <end>',
 '<start> i cannot sleep i cannot dream tonight <end>',
 '<start> i need somebody and always <end>',
 '<start> this sick strange darkness <end>']

###**[번외]기타 실험**

In [5]:
#<번외-비교 대상군>
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

    #잘 적용되었는지 확인 / 단어 15개 넘는 문장 찾는다고 노가다함ㅠㅠ
corpus[140:150]


['<start> don t waste your time on me you re already <end>',
 '<start> the voice inside my head i miss you , miss you <end>',
 '<start> don t waste your time on me you re already <end>',
 '<start> the voice inside my head i miss you , miss you <end>',
 '<start> don t waste your time on me you re already <end>',
 '<start> the voice inside my head i miss you , miss you <end>',
 '<start> i miss you , miss you <end>',
 '<start> i miss you , miss you <end>',
 '<start> i miss you , miss you <end>',
 '<start> i miss you , miss you i miss you miss you hello there the angel from my nightmare <end>']

In [7]:
#<번외-len()1>
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
      #토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외
    if sentence[-1] == ":": continue
    if len(sentence) > 15: continue

    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

    #잘 적용되었는지 확인
corpus[140:150]

['<start> all around you <end>',
 '<start> that he s in <end>',
 '<start> to a shadow <end>',
 '<start> and i wonder <end>',
 '<start> silvio <end>',
 '<start> silver and gold <end>',
 '<start> silvio <end>',
 '<start> i gotta go <end>',
 '<start> silvio <end>',
 '<start> silver and gold <end>']

In [ ]:
#<번외-len()2>
orpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
      #토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외
    if sentence[-1] == ":": continue
    if len(sentence.split()) > 15: continue

    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

    #잘 적용되었는지 확인
corpus[140:150]

['<start> ok a millie sold first day i went gold how do i celebrate work on the carter <end>',
 '<start> yup i aint here to brag nor boast this is simply an attempt to thank you <end>',
 '<start> the most <end>',
 '<start> you the fan you the man and to my female audience i hope you use sanitizer <end>',
 '<start> cuz im kissin all ya hands <end>',
 '<start> all my plans is well executed <end>',
 '<start> this that electric music you can get electrocuted <end>',
 '<start> you know i extra do it they say im the best to do <end>',
 '<start> i say im better than who next to do it or whoever do it <end>',
 '<start> they could never do it like me i c o n or you could call me mr i go in <end>']

❗ 토큰 15개 이상 컷이 잘 진행되었는지 확인을 위해 실험   
1번째부터 139번째까지 단어 15개 이상의 문장이 없어서 노가다로 단어 15개 이상의 문장이 나올 때까지 찾아봄   
그리하여 corpus[140:150] 으로 코드 비교를 해봄   
처음에는 counter를 사용하여 컷하려고 했으나 아직 사용법이 미숙하여 계속 오류가 남 -> 찾아보았으나 완벽히 이해는 하지 못함   
그래서 가장 통상적으로 많이 사용하는 len을 이용함 -> 처음에는 별 문제가 없어보였으나 corpus[140:150]을 통해 문장을 확인해보니 문장이 이상하게 잘림을 발견
그래서 여러 방법을 시도한 후 count 로 가장 적합하게 컷을 함.

##Step 4. 평가 데이터셋 분리
### **토크나이저 생성**

: 문장을 일정한 기준으로 쪼개는 과정

- 문장의 최대길이를 15로 설정
- 최대길이가 15보다 작으면 padding값으로 대체(padding=0)
- 최대길이가 15보다 크면 뒷 부분 제거

In [76]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, #단어장의 개수
        filters=' ',    #쓰지 않겠다. 위에서 한 번 정제했기 때문에 필터는 사실상 필요가 없다, 애초에 전처리를 위해서 필요한 아이
        oov_token="<unk>"    #12000단어에 포함되지 못한 단어는 '<unk>'로 바꾸기
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2   4 375 ...   0   0   0]
 [  2   4  35 ...   0   0   0]
 [  2 106  39 ...   0   0   0]
 ...
 [  2   3   0 ...   0   0   0]
 [  2   3   0 ...   0   0   0]
 [  2   3   0 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7ff31c58bc90>


### **텐서플로우 활용 데이터 분리**

In [77]:
src_input = tensor[:, :-1]   # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성

print(src_input[0])
print(tgt_input[0])

[  2   4 375  16 251  10 152   7  15   3   0   0   0   0]
[  4 375  16 251  10 152   7  15   3   0   0   0   0   0]


In [78]:
#train과 Test(val) 분리

from sklearn.model_selection import train_test_split
# lms 지시
#총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)
#섞을 이유가 없기 때문에 random_state는 사용하지 않겠다.

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (128205, 14)
Target Train: (128205, 14)


In [79]:
#데이터셋 정의하기, 객체 생성

BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 64
steps_per_epoch = len(enc_train) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 14), (64, 14)), types: (tf.int32, tf.int32)>

### Step 5. 인공지능 만들기

In [80]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024    #단어 하나의 특징 수
hidden_size = 2048    #퍼셉트론의 개수
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)


for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(64, 14, 12001), dtype=float32, numpy=
array([[[ 4.4040549e-05,  4.0495541e-04, -1.2976209e-04, ...,
         -8.5205495e-05, -8.3998137e-05, -3.6425921e-04],
        [ 1.1136556e-04,  3.2280807e-04, -5.1000941e-04, ...,
         -4.6400796e-04, -3.4815349e-04, -6.4714014e-04],
        [ 6.4323033e-04, -1.0870084e-04, -7.2515791e-04, ...,
         -9.9677290e-04, -5.0296797e-04, -1.2085245e-03],
        ...,
        [ 4.5412159e-04, -3.0151508e-03,  1.4298427e-03, ...,
         -2.0543844e-03,  5.2642409e-04, -2.6084161e-03],
        [-3.5977871e-05, -3.5673396e-03,  2.3289821e-03, ...,
         -2.5781200e-03,  9.7355398e-04, -2.4842490e-03],
        [-4.9672951e-04, -4.0722392e-03,  3.0743345e-03, ...,
         -3.0731484e-03,  1.4088321e-03, -2.3290243e-03]],

       [[ 4.4040549e-05,  4.0495541e-04, -1.2976209e-04, ...,
         -8.5205495e-05, -8.3998137e-05, -3.6425921e-04],
        [ 2.3893047e-04,  4.5186878e-04, -3.7188581e-04, ...,
          4.4418457e-06,  

In [81]:
model.summary()

Model: "text_generator_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    multiple                  12289024  
                                                                 
 lstm_22 (LSTM)              multiple                  25174016  
                                                                 
 lstm_23 (LSTM)              multiple                  33562624  
                                                                 
 dense_11 (Dense)            multiple                  24590049  
                                                                 
Total params: 95,615,713
Trainable params: 95,615,713
Non-trainable params: 0
_________________________________________________________________


###**최종 모델 학습(🎉🎉완벽 성공🎉🎉)**

In [82]:
# 아홉 번째 시도한 모델 학습
# num_words 12000, batch_size=64
# embedding_size = 1024
#hidden_size = 2048

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train,epochs=5, batch_size=64, validation_data=(enc_val, dec_val))

Epoch 1/5
2004/2004 [==============================] - 362s 179ms/step - loss: 3.0385 - val_loss: 2.7356
Epoch 2/5
2004/2004 [==============================] - 356s 178ms/step - loss: 2.4752 - val_loss: 2.4634
Epoch 3/5
2004/2004 [==============================] - 367s 183ms/step - loss: 2.0433 - val_loss: 2.3017
Epoch 4/5
2004/2004 [==============================] - 367s 183ms/step - loss: 1.6700 - val_loss: 2.2180
Epoch 5/5
2004/2004 [==============================] - 356s 178ms/step - loss: 1.3867 - val_loss: 2.1995


###**validation loss 2.2 이하를 위한 노력의 흔적**
아래 부분은 모델 학습도 기록하고자 그대로 두었습니다.   
패스하셔도 됩니다.

In [18]:
# 첫 번째 시도한 모델 학습
# num_words 12000, batch_size=512
#embedding_size = 256
#hidden_size = 1024

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train,epochs=10, batch_size=512, validation_data=(enc_val, dec_val))

Epoch 1/10
251/251 [==============================] - 156s 606ms/step - loss: 2.3093 - val_loss: 2.5962
Epoch 2/10
251/251 [==============================] - 159s 633ms/step - loss: 2.2174 - val_loss: 2.5835
Epoch 3/10
251/251 [==============================] - 151s 601ms/step - loss: 2.1552 - val_loss: 2.5713
Epoch 4/10
251/251 [==============================] - 151s 601ms/step - loss: 2.0977 - val_loss: 2.5628
Epoch 5/10
251/251 [==============================] - 159s 633ms/step - loss: 2.0426 - val_loss: 2.5578
Epoch 6/10
251/251 [==============================] - 151s 601ms/step - loss: 1.9896 - val_loss: 2.5499
Epoch 7/10
251/251 [==============================] - 159s 633ms/step - loss: 1.9385 - val_loss: 2.5458
Epoch 8/10
251/251 [==============================] - 151s 600ms/step - loss: 1.8891 - val_loss: 2.5406
Epoch 9/10
251/251 [==============================] - 151s 601ms/step - loss: 1.8407 - val_loss: 2.5374
Epoch 10/10
251/251 [==============================] - 151s 600m

In [17]:
# 두 번째 시도한 모델 학습
# num_words 12000, batch_size=256
#embedding_size = 256
#hidden_size = 1024

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train,epochs=10, batch_size=256, validation_data=(enc_val, dec_val))

Epoch 1/10
501/501 [==============================] - 100s 193ms/step - loss: 3.5468 - val_loss: 3.1800
Epoch 2/10
501/501 [==============================] - 97s 194ms/step - loss: 3.0625 - val_loss: 2.9899
Epoch 3/10
501/501 [==============================] - 99s 197ms/step - loss: 2.8831 - val_loss: 2.8722
Epoch 4/10
501/501 [==============================] - 99s 197ms/step - loss: 2.7470 - val_loss: 2.7837
Epoch 5/10
501/501 [==============================] - 99s 197ms/step - loss: 2.6317 - val_loss: 2.7185
Epoch 6/10
501/501 [==============================] - 97s 194ms/step - loss: 2.5242 - val_loss: 2.6629
Epoch 7/10
501/501 [==============================] - 99s 198ms/step - loss: 2.4239 - val_loss: 2.6156
Epoch 8/10
501/501 [==============================] - 98s 195ms/step - loss: 2.3296 - val_loss: 2.5795
Epoch 9/10
501/501 [==============================] - 97s 195ms/step - loss: 2.2406 - val_loss: 2.5417
Epoch 10/10
501/501 [==============================] - 97s 194ms/step - 

In [22]:
# 세 번째 시도한 모델 학습
# num_words 12000, batch_size=128
#embedding_size = 256
#hidden_size = 1024

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train,epochs=10, batch_size=128, validation_data=(enc_val, dec_val))

Epoch 1/10
1002/1002 [==============================] - 109s 106ms/step - loss: 3.4206 - val_loss: 3.1017
Epoch 2/10
1002/1002 [==============================] - 106s 106ms/step - loss: 2.9791 - val_loss: 2.9133
Epoch 3/10
1002/1002 [==============================] - 107s 106ms/step - loss: 2.7883 - val_loss: 2.7936
Epoch 4/10
1002/1002 [==============================] - 107s 107ms/step - loss: 2.6361 - val_loss: 2.7075
Epoch 5/10
1002/1002 [==============================] - 105s 105ms/step - loss: 2.4985 - val_loss: 2.6401
Epoch 6/10
1002/1002 [==============================] - 105s 105ms/step - loss: 2.3700 - val_loss: 2.5883
Epoch 7/10
1002/1002 [==============================] - 107s 107ms/step - loss: 2.2486 - val_loss: 2.5471
Epoch 8/10
1002/1002 [==============================] - 106s 106ms/step - loss: 2.1351 - val_loss: 2.5155
Epoch 9/10
1002/1002 [==============================] - 106s 106ms/step - loss: 2.0282 - val_loss: 2.4908
Epoch 10/10
1002/1002 [=======================

In [31]:
# 네 번째 시도한 모델 학습
# num_words 15000, batch_size=128
#embedding_size = 256
#hidden_size = 1024

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train,epochs=10, batch_size=128, validation_data=(enc_val, dec_val))

Epoch 1/10
1002/1002 [==============================] - 117s 114ms/step - loss: 3.4107 - val_loss: 3.1122
Epoch 2/10
1002/1002 [==============================] - 123s 123ms/step - loss: 2.9958 - val_loss: 2.9346
Epoch 3/10
1002/1002 [==============================] - 114s 114ms/step - loss: 2.8168 - val_loss: 2.8257
Epoch 4/10
1002/1002 [==============================] - 113s 112ms/step - loss: 2.6769 - val_loss: 2.7475
Epoch 5/10
1002/1002 [==============================] - 112s 112ms/step - loss: 2.5519 - val_loss: 2.6865
Epoch 6/10
1002/1002 [==============================] - 113s 113ms/step - loss: 2.4359 - val_loss: 2.6382
Epoch 7/10
1002/1002 [==============================] - 123s 123ms/step - loss: 2.3278 - val_loss: 2.6004
Epoch 8/10
1002/1002 [==============================] - 124s 124ms/step - loss: 2.2279 - val_loss: 2.5712
Epoch 9/10
1002/1002 [==============================] - 124s 124ms/step - loss: 2.1346 - val_loss: 2.5506
Epoch 10/10
1002/1002 [=======================

In [49]:
# 다섯 번째 시도한 모델 학습
# num_words 12000, batch_size=128
# 옵티마이저 수정
#embedding_size = 128
#hidden_size = 1024

optimizer = tf.keras.optimizers.SGD()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10, batch_size=128, validation_data=(enc_val, dec_val))

Epoch 1/10
1002/1002 [==============================] - 94s 90ms/step - loss: 85.3905 - val_loss: 60.1697
Epoch 2/10
1002/1002 [==============================] - 92s 91ms/step - loss: 52.2011 - val_loss: 45.1717
Epoch 3/10
1002/1002 [==============================] - 90s 90ms/step - loss: 47.8096 - val_loss: 42.3885
Epoch 4/10
1002/1002 [==============================] - 92s 91ms/step - loss: 44.4562 - val_loss: 55.8165
Epoch 5/10
1002/1002 [==============================] - 90s 90ms/step - loss: 48.0046 - val_loss: 32.7403
Epoch 6/10
1002/1002 [==============================] - 92s 91ms/step - loss: 55.3751 - val_loss: 52.3362
Epoch 7/10
1002/1002 [==============================] - 90s 90ms/step - loss: 52.6024 - val_loss: 67.0664
Epoch 8/10
1002/1002 [==============================] - 90s 90ms/step - loss: 53.5306 - val_loss: 54.2085
Epoch 9/10
1002/1002 [==============================] - 90s 90ms/step - loss: 51.0313 - val_loss: 31.8335
Epoch 10/10
1002/1002 [=======================

In [57]:
# 여섯 번째 시도한 모델 학습
# num_words 12000, batch_size=128
# embedding_size = 512
#hidden_size = 1024

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train,epochs=10, batch_size=128, validation_data=(enc_val, dec_val))

Epoch 1/10
1002/1002 [==============================] - 115s 112ms/step - loss: 3.3277 - val_loss: 3.0330
Epoch 2/10
1002/1002 [==============================] - 112s 112ms/step - loss: 2.9015 - val_loss: 2.8401
Epoch 3/10
1002/1002 [==============================] - 112s 112ms/step - loss: 2.7029 - val_loss: 2.7215
Epoch 4/10
1002/1002 [==============================] - 112s 112ms/step - loss: 2.5397 - val_loss: 2.6364
Epoch 5/10
1002/1002 [==============================] - 112s 112ms/step - loss: 2.3929 - val_loss: 2.5717
Epoch 6/10
1002/1002 [==============================] - 112s 112ms/step - loss: 2.2566 - val_loss: 2.5195
Epoch 7/10
1002/1002 [==============================] - 112s 112ms/step - loss: 2.1298 - val_loss: 2.4821
Epoch 8/10
1002/1002 [==============================] - 112s 112ms/step - loss: 2.0117 - val_loss: 2.4535
Epoch 9/10
1002/1002 [==============================] - 112s 112ms/step - loss: 1.9011 - val_loss: 2.4292
Epoch 10/10
1002/1002 [=======================

In [60]:
# 일곱 번째 시도한 모델 학습
# num_words 12000, batch_size=128
# embedding_size = 1024
#hidden_size = 1024

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train,epochs=15, batch_size=128, validation_data=(enc_val, dec_val))

Epoch 1/15
1002/1002 [==============================] - 130s 128ms/step - loss: 3.2690 - val_loss: 2.9727
Epoch 2/15
1002/1002 [==============================] - 118s 118ms/step - loss: 2.8378 - val_loss: 2.7813
Epoch 3/15
1002/1002 [==============================] - 118s 118ms/step - loss: 2.6278 - val_loss: 2.6580
Epoch 4/15
1002/1002 [==============================] - 118s 118ms/step - loss: 2.4426 - val_loss: 2.5676
Epoch 5/15
1002/1002 [==============================] - 118s 118ms/step - loss: 2.2695 - val_loss: 2.4959
Epoch 6/15
1002/1002 [==============================] - 119s 119ms/step - loss: 2.1058 - val_loss: 2.4366
Epoch 7/15
1002/1002 [==============================] - 119s 118ms/step - loss: 1.9511 - val_loss: 2.3969
Epoch 8/15
1002/1002 [==============================] - 119s 119ms/step - loss: 1.8084 - val_loss: 2.3710
Epoch 9/15
1002/1002 [==============================] - 119s 119ms/step - loss: 1.6771 - val_loss: 2.3524
Epoch 10/15
1002/1002 [=======================

In [67]:
# 여덟 번째 시도한 모델 학습
# num_words 12000, batch_size=64
# embedding_size = 1024
#hidden_size = 1024

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train,epochs=10, batch_size=64, validation_data=(enc_val, dec_val))

Epoch 1/10
2004/2004 [==============================] - 135s 66ms/step - loss: 3.1336 - val_loss: 2.8551
Epoch 2/10
2004/2004 [==============================] - 132s 66ms/step - loss: 2.6900 - val_loss: 2.6507
Epoch 3/10
2004/2004 [==============================] - 132s 66ms/step - loss: 2.4159 - val_loss: 2.5147
Epoch 4/10
2004/2004 [==============================] - 132s 66ms/step - loss: 2.1670 - val_loss: 2.4227
Epoch 5/10
2004/2004 [==============================] - 132s 66ms/step - loss: 1.9406 - val_loss: 2.3612
Epoch 6/10
2004/2004 [==============================] - 132s 66ms/step - loss: 1.7386 - val_loss: 2.3271
Epoch 7/10
2004/2004 [==============================] - 132s 66ms/step - loss: 1.5664 - val_loss: 2.3178
Epoch 8/10
2004/2004 [==============================] - 132s 66ms/step - loss: 1.4221 - val_loss: 2.3198
Epoch 9/10
2004/2004 [==============================] - 132s 66ms/step - loss: 1.3040 - val_loss: 2.3369
Epoch 10/10
2004/2004 [==============================] 

❗ 밸리데이션 로스를 줄이기 위한 노력은 아래 회고에서 다루겠다.

❗ 밸리데이션 로스   
1) 밸리데이션 로스란?  
overfitting을 해결하기 위해 별로도 만들어진 dataset이 아니라, training dataset에서 추출된 가상의 dataset임.

2) 만약 밸리데이션 로스가 안 줄어들 땐?   
- 데이터 전처리 : 데이터 표준화 및 정규화(배치놈, 스케일링)   
- 모델 강제성 : 모델이 너무 복잡하지 않은지 확인, dropout을 추가하고 각 계층의 레이어 수 또는 뉴런 수 줄이기   
- 학습 속도 및 감소 속도 : 학습 속도 줄이기

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbhzwwY%2FbtqAtbHdeNA%2FpRBnbKySV9asFqpI1Ozc71%2Fimg.png" width="" height=""  title="px(픽셀) 크기 설정" alt="밸리데이션데이터셋"></img><br/>
[밸리데이션 데이터셋을 이용한 학습 과정]
만약 밸리데이션 로스가 증가했다면 학습을 종료한다.   
그렇지 않을 경우는 (2)로 돌아가 학습을 계속 진행함   

(3)(4)과정을 통해 현재 모델이 학습 과정에서 참조하지 않았던 data를 얼마나 정확하게 예측하는지 평가하고, 이를 학습의 종료 조건으로 이용함으로써 overfitting을 간접적으로 방지
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb7W55F%2FbtqAtCq1C4B%2FifxO1FuK1p0b3LuzvJKbOk%2Fimg.png" width="" height=""  title="px(픽셀) 크기 설정" alt="밸리데이션로스"></img><br/>
validation loss가 증가하는 시점부터 overfitting이 발생했다고 판단하고, 이에 따라 학습을 중단한다.   
 validation dataset 또한 test dataset을 완벽히 표현하지는 못 하기 때문에 validation loss가 최소가 되는 시점이 test loss가 최소가 되는 시점과 정확히 일치하지는 않을 수도 있다.

##Step 6. 작문 평가 

In [63]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

이는 모델 학습 실험 과정 중에 받아낸 결과값이다.   
제일 처음 받았던 문장은 다소 괴상했다(?) 아무생각없이 지워버려 기록이 없지만..ㅠㅠ
그래서 우선 최종 모델 학습 결과와 비교를 하기 위해 남겨두었다.   
validation loss 근방에서는 문장이 위와 동일하게 출력되었다.

In [85]:
keywords = ['I love', 'you are', 'love', 'I', 'she', 'he', 'If']
for start in keywords:
    print(generate_text(model, tokenizer, init_sentence= ' '.join(["<start>", start])))

<start> i love you , i love you <end> 
<start> you are gonna have to find out for yourself <end> 
<start> love is a beautiful thing <end> 
<start> i m gonna make contact tonight . <end> 
<start> she s got that vibe <end> 
<start> he s a monster <end> 
<start> if you want me <end> 


 마지막 모델 학습 완료 후 출력해보았다.   
 I love에 대해서는 'i love you , i love you' 가 출력되었다.   
 그 외 다른 키워드도 제시해보았다. 꽤나 완성도 높은 문장들이 출력되었다.   
이 정도 결과물이면 훌륭하다고 생각하기에, 나는 매우 만족한다.

#Step 7. 회고
<정제함수 활용 데이터 구축>

토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외 코드를 짜줄 때 여러 오류가 발생했다

1. if len(sentence.count()) >15: continue

  => TypeError: count() takes at least 1 argument (0 given)

  인수가 필요하다길래 count() 안에 corpus 를 넣어주었다.
2. if len(sentence.count(corpus)) >15: continue

  => TypeError: must be str, not list

여기서 아직도 개념이 잘 잡히지 않은 것 같아 파이썬 '인수'가 무엇인가? 
  => 함수를 정의할 때 넣는 값(변수)를 매개변수라 하고, 함수를 호출할 때 넣는 값을 인수라 한다.

  여기서 감이 잡혔다. 함수를 호출할 때 넣는 값을 인수라고 하는데 그럼 해당 부분은 preprocessed_sentence가 인수이니까

  3. if len(preprocessed_sentence.count( )) >15: continue
  
  으음... 이것도 해결이 안되는거 같은데 count 함수는 못 쓰나..?


4. from collections import Counter    #count 사용을 위해 import 하기

5. if Counter(sentence) >15: continue
  => TypeError: '>' not supported between instances of 'Counter' and 'int'

   세상에 부등호를 지원안한다니ㅠㅠ
   <= 또는 >= 이렇게 사용해아한다고 한다..

   ** counter() 와 count()의 차이

###❗❗밸리데이션 로스❗❗
이번 모델에서 핵심은 밸리데이션 로스를 2.2 이하로 줄이는 것이다.   
이를 줄이기 위해 상당히 애썼는데 한 번 구현할 때마다 시간이 상당히 오래걸려 쉽지 않았다.   
처음에 건드린건 배치사이즈였다.

- ⭐ Batch size란?   
연산 한 번 할때 들어가는 데이터의 크기를 말함.   
배치사이즈가 큰 경우(숫자가 작을 경우) 한 번에 처리해야 할 데이터의 양이 많아지기 때문에 학습 속도가 느려짐.  
배치사이즈가 작을 경우(숫자가 큼) 적은 데이터로 업데이트가 자주 발생해서 훈련이 불안정함  


(1) 처음에는 batch_size=512로 시작하였다. 초반부터 validation loss가 꽤나 좋게 나와 기대했지만 2.5 부근에서 고전을 면하지 못했다. 그래서 batch_size=256 그리고 batch_size=128 로 변경하여 진행해보았다. batch_size=256로 하였을 때 validation loss값이 점점 좋아지는 것이 눈에 띄게 보였다. 하지만 역시나 2.5 부근에서는 validation loss이 거의 제자리 걸음이었다. batch_size=128로 하였을 땐 한 번에 처리해야할 데이터가 1000이상이 되면서 epoch 한번 도는데 상당한 시간이 소요되었다. 하지만 2.5이하의 의미있는 validation loss값을 발견하였고, batch_size는 128로 고정시키고 다른 요소를 건드렸다. 여기서 처음부터 epoch 수치를 건들지 않은 이유는 epoch 반복할 때마다 나오는 validation loss 수치에서 의미있는 하락폭이 나오지 않으면 아무리 epoch를 높여봤자 원하는 수치를 얻기에는 힘들 것이라고 판단하였고, 과접합 문제를 생각해서라도 마지막에 수정하는 것이 좋을 것이라고 생각했다.   
(2) 두 번째로 만진 건 num_words 이다. 12000 이상으로 사용하라는 기준이 있었기에 혹시나 학습 데이터가 작아서 생긴 문제지 않을까 싶어 15000으로 늘려 학습시켜보았다. 단어장 개수 15000개가 12000개보다 loss값이 안 좋아 12000개로 고정시켰다.   
(3) 세 번째로는 optimizer를 수정해보았다. Adam 에서 SGD로 변경해보았는데 loss값이 갑자기 두자릿수가 나와 다시 Adam으로 돌렸다...
- 옵티마이저 Adam
Adam 기법은 momentum 기법과 Rmsprop 기법을 혼합한 기법으로 주로 다른 옵티마이저와 성능비교를 해보면 손실값이 가장 많이 떨어지는 것을 확인할 수 있다.
- 옵티마이저 SGD
확률적 경사하강법으로, SGD는 Adam과 반대로 성능이 가장 좋지 않은 경우를 많이 보여준다. SGD는 손실함수의 기울기가 무작정 낮아지는 방향으로 진행되기 때문에 손실함수의 기울기가 최저값이 되는 값을 찾는데 비효율적인 탐색경로를 지니게 된다.

(4) 네 번째로 embedding_size를 수정함. embedding_size가 커질수록 Total params 가 늘어났고, 이는 학습에 긍정적인 영향을 끼칠것이라 생각했다. 실제로 embedding_size가 커질수록 validation loss 값이 더욱 좋아지는 것을 발견. validation loss = 128에서 256, 512, 1024까지 늘려서 학습을 시킴.   
(5) 다섯 번째는 epoch 를 15로 늘려서 실험하였으나 이상하게도 validation loss가 점점 떨어지더니 다시 증가하였다. 바로 오버피팅이었던 것이다. 그래서 epoch=10이 가장 적당한 학습횟수라고 판단하였다. 이에 batch_size을 재수정하고 학습을 시켜보았다.   
(6) 위 과정을 수행하였음에도 불구하고 validation loss가 2.2 이하에 도달하지 못하였다. 따라서 hidden_size를 2배로 적용시켜주었다. 이렇게 설정해주니 epoch 6에서부터 validation loss 가 다시 커졌다. 즉 오버피팅이 일어난 것이다.   
(7) 최종적으로 epoch 5로 설정하여  validation loss를 완벽하게 2.2 이하로 도출해냈다!!!!!!!!!!

#Step 8. Reference
1) https://untitledtblog.tistory.com/158   
2) https://choosunsick.github.io/post/optimizer_compare/